# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import re
import time
import pandas as pd
import google.generativeai as genai

In [2]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = 'Sentiment_Analysis_Results.csv'

# Import Data

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('../02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('../02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('../02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('../02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10126, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol: COP ...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10121,Earnings Call Q&A,EQ-338,XOM,Feb-02-2021,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\r\nOperator\r\n[Operator I...,NaN
10122,Earnings Call Q&A,EQ-339,COP,Feb-02-2021,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\r\nOperator\r\n[Operator I...,NaN
10123,Earnings Call Q&A,EQ-340,EOG,May-03-2019,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\r\nOperator\r\n[Operator I...,NaN
10124,Earnings Call Q&A,EQ-341,SHEL,May-02-2019,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\r\nOperator\r\n[Operator I...,NaN
10125,Earnings Call Q&A,EQ-342,COP,Apr-30-2019,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\r\nOperator\r\n[Operator I...,NaN


# Find or Create Results CSV

In [4]:
# Determine if the Sentiment Analysis Results file already exists
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")

The file exists in the current directory.


In [5]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy text df and drop the text and headline column due to size
    empty_sentiment_df = text_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""
        empty_sentiment_df[category] = empty_sentiment_df[category].astype('object')

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

### NOTE: If you want to re-run the sentiment analysis, delete or archive the csv to create a blank one

# Sentiment Analysis
NOTE: Google gemini **currently** has a daily query limit of 1,500 requests per day.  As we have over 10,000 documents, the code will be designed to run over multiple days and pick up where we last left off.  After the code is run, the user will still need to manually download the csv and upload to github.

**Gemini Free Rate Limits**
*   15 RPM (requests per minute)
*   1 million TPM (tokens per minute)
*   1,500 RPD (requests per day)

In [6]:
# Set up Gemini. (API key needs to be in your environment variables)
key = 'GOOGLE_API_KEY'
GOOGLE_API_KEY = os.getenv(key)
genai.configure(api_key=GOOGLE_API_KEY)

# So it doesn't block the output
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",},]

model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safety_settings)

In [7]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

IR-3288


In [8]:
# Helper function to get Gemini query function inputs
def get_gemini_inputs(text_df, unique_id):
    """
    Retrieves information from the DataFrame based on the unique ID and outputs company, source, headline, and text.

    Args:
        text_df (pd.DataFrame): The DataFrame containing the text data.
        unique_id (str): The unique ID to search for.

    Returns:
        tuple: A tuple containing company, source, headline, and text.
    """
    # Find the row with the specified unique ID
    row = text_df[text_df['Unique_ID'] == unique_id]

    # Extract the required information
    company = row['Ticker'].values[0]
    source = row['Source'].values[0]
    headline = row['Article Headline'].values[0]
    text = row['Article Text'].values[0]

    return company, source, headline, text

# Test function
company, source, headline, text = get_gemini_inputs(text_df, unique_id)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: OXY

Source: Investment Research

Headline: Occidental Petroleum Corporation

Text:
Stock Report | August 27, 2022 | NYSE Symbol: OXY | OXY is in the S&P 500
Occidental Petroleum Corporation
Recommendation Price 12-Mo. Target Price Report Currency Investment Style
BUY « « « « «
USD 73.55 (as of market close Aug 26, 2022) USD 73.00 USD Large-Cap Value
Equity Analyst Stewart Glickman, CFA
GICS Sector Energy Summary One of the largest oil & gas companies in the U.S., OXY has global exploration & production
Sub-Industry Integrated Oil and Gas operations and a large chemicals business.
Key Stock Statistics (Source: CFRA, S&P Global Market Intelligence (SPGMI), Company Reports)
52-Wk Range USD 76.1 - 23.91 Oper.EPS2022E USD 10.70 Market Capitalization[B] USD 68.33 Beta 1.91
Trailing 12-Month EPS USD 7.63 Oper.EPS2023E USD 8.35 Yield [%] 0.71 3-yr Proj. EPS CAGR[%] 22
Trailing 12-Month P/E 9.64 P/E on Oper.EPS2022E 6.87 Dividend Rate/Share USD 0.52 SPGMI's Quality Ranking B-
USD 10K 

In [9]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test function
response = query_gemini(company, source, headline, text, model)
print(response)

Here's a breakdown of the sentiment analysis for the OXY analyst report, along with reasoning and references to the text:

**Finance:**

- **Sentiment: Positive** 
- **Reasoning:**  The report highlights OXY's efforts to reduce debt (mentioning the $6.7 billion repayment in 2021 and the goal of reaching $20 billion in net debt) and expresses optimism about further progress due to strong oil prices. The report also mentions OXY's dividend increase and potential for share buybacks, which are positive for investors.  [See pages 1, 3, 4, and 5 of the report]

**Production:**

- **Sentiment: Neutral** 
- **Reasoning:** The report acknowledges that OXY is aiming for relatively flat upstream production in 2022, citing higher oil service costs and inflation as contributing factors. The report also mentions a joint venture with Ecopetrol that will add to production plans but does not specify a clear positive or negative impact on overall production. [See page 1, 4, and 5]

**Reserves / Explorat

In [10]:
# Function to parse text
def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        # Create a regex pattern to match the format "- Category - Sentiment"
        # The category name is escaped to handle any special characters
        pattern = rf"- {re.escape(category)} - (\w+)"

        # Search for the pattern in the text
        match = re.search(pattern, text)

        # If a match is found, extract the sentiment and add it to the results dictionary
        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

# Test function
fail_text = """
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Bad
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral
"""

sentiment_dict = parse_sentiment(response, CATEGORIES)
fail_sentiment = parse_sentiment("fail_text", CATEGORIES)
print(sentiment_dict)
print(fail_sentiment)

{'Finance': 'Positive', 'Production': 'Neutral', 'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Negative', 'Environment / Regulatory / Geopolitics': 'Neutral', 'Alternative Energy / Lower Carbon': 'Neutral', 'Oil Price / Natural Gas Price / Gasoline Price': 'Positive'}
Did not find all categories


In [11]:
# Function to update the csv
def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Find the index of the row with the specified unique ID
    row_index = df[df['Unique_ID'] == unique_id].index

    # Update the columns based on the sentiment dictionary
    for category, sentiment in sentiment_dict.items():
        df.loc[row_index, category] = sentiment

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

# Test function
update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

Row with Unique_ID 'IR-3288' has been updated.


In [12]:
start_time = time.time()
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
count = 0
max_tries = 5

# Iterate through the CSV using the functions
while unique_id:
    retries = 0
    success = False

    # There are multiple errors that can happen, many of them simply need another try
    while retries < max_tries and not success:
        try:
            # Get gemini inputs
            company, source, headline, text = get_gemini_inputs(text_df, unique_id)

            # Query Gemini
            response = query_gemini(company, source, headline, text, model)

            # Parse text
            sentiment_dict = parse_sentiment(response, CATEGORIES)

            # Update the csv
            update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

            success = True

        except Exception as e:
            retries += 1
            print(f"Error encountered: {e}. Retry {retries}/{max_tries}")

            if retries >= max_tries:
                print(f"Failed to process unique_id {unique_id} after {max_tries} attempts. Stopping.")
                # Exit both loops
                unique_id = None
                break

    if not success:
        break

    # Get an update every 10 rows
    count += 1
    if count % 10 == 0:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Test print statements
    # print(unique_id)
    # print(f"Company: {company}\n")
    # print(f"Source: {source}\n")
    # print(f"Headline: {headline}\n")
    # print(f"Text:\n{text}")
    # print(response)
    # print(sentiment_dict)

Row with Unique_ID 'IR-3289' has been updated.
Row with Unique_ID 'IR-3290' has been updated.
Row with Unique_ID 'IR-3291' has been updated.
Row with Unique_ID 'IR-3292' has been updated.
Row with Unique_ID 'IR-3293' has been updated.
Row with Unique_ID 'IR-3294' has been updated.
Row with Unique_ID 'IR-3295' has been updated.
Row with Unique_ID 'IR-3296' has been updated.
Row with Unique_ID 'IR-3297' has been updated.
Row with Unique_ID 'IR-3298' has been updated.
Iteration: 10, Elapsed Time: 1 minutes and 49.54 seconds
Row with Unique_ID 'IR-3299' has been updated.
Row with Unique_ID 'IR-3300' has been updated.
Row with Unique_ID 'IR-3301' has been updated.
Row with Unique_ID 'IR-3302' has been updated.
Row with Unique_ID 'IR-3303' has been updated.
Row with Unique_ID 'IR-3304' has been updated.
Row with Unique_ID 'IR-3305' has been updated.
Row with Unique_ID 'IR-3306' has been updated.
Row with Unique_ID 'IR-3307' has been updated.
Row with Unique_ID 'IR-3308' has been updated.
Ite

Row with Unique_ID 'IR-3424' has been updated.
Row with Unique_ID 'IR-3425' has been updated.
Row with Unique_ID 'IR-3426' has been updated.
Row with Unique_ID 'IR-3427' has been updated.
Row with Unique_ID 'IR-3428' has been updated.
Row with Unique_ID 'IR-3429' has been updated.
Iteration: 140, Elapsed Time: 96 minutes and 39.29 seconds
Row with Unique_ID 'IR-3430' has been updated.
Row with Unique_ID 'IR-3431' has been updated.
Row with Unique_ID 'IR-3432' has been updated.
Row with Unique_ID 'IR-3433' has been updated.
Row with Unique_ID 'IR-3434' has been updated.
Row with Unique_ID 'IR-3435' has been updated.
Row with Unique_ID 'IR-3436' has been updated.
Row with Unique_ID 'IR-3437' has been updated.
Row with Unique_ID 'IR-3438' has been updated.
Row with Unique_ID 'IR-3439' has been updated.
Iteration: 150, Elapsed Time: 98 minutes and 25.36 seconds
Row with Unique_ID 'IR-3440' has been updated.
Row with Unique_ID 'IR-3441' has been updated.
Error encountered: 'str' object has 

Row with Unique_ID 'IR-3571' has been updated.
Row with Unique_ID 'IR-3572' has been updated.
Row with Unique_ID 'IR-3573' has been updated.
Row with Unique_ID 'IR-3574' has been updated.
Row with Unique_ID 'IR-3575' has been updated.
Row with Unique_ID 'IR-3576' has been updated.
Row with Unique_ID 'IR-3577' has been updated.
Row with Unique_ID 'IR-3578' has been updated.
Row with Unique_ID 'IR-3579' has been updated.
Iteration: 290, Elapsed Time: 150 minutes and 45.23 seconds
Row with Unique_ID 'IR-3580' has been updated.
Row with Unique_ID 'IR-3581' has been updated.
Row with Unique_ID 'IR-3582' has been updated.
Row with Unique_ID 'IR-3583' has been updated.
Row with Unique_ID 'IR-3584' has been updated.
Row with Unique_ID 'IR-3585' has been updated.
Row with Unique_ID 'IR-3586' has been updated.
Row with Unique_ID 'IR-3587' has been updated.
Row with Unique_ID 'IR-3588' has been updated.
Row with Unique_ID 'IR-3589' has been updated.
Iteration: 300, Elapsed Time: 152 minutes and 2

Row with Unique_ID 'IR-3713' has been updated.
Row with Unique_ID 'IR-3714' has been updated.
Row with Unique_ID 'IR-3715' has been updated.
Row with Unique_ID 'IR-3716' has been updated.
Row with Unique_ID 'IR-3717' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-3718' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-3719' has been updated.
Iteration: 430, Elapsed Time: 210 minutes and 37.58 seconds
Row with Unique_ID 'IR-3720' has been updated.
Row with Unique_ID 'IR-3721' has been updated.
Row with Unique_ID 'IR-3722' has been updated.
Row with Unique_ID 'IR-3723' has been updated.
Row with Unique_ID 'IR-3724' has been updated.
Row with Unique_ID 'IR-3725' has been updated.
Row with Unique_ID 'IR-3726' has been updated.
Row with Unique_ID 'IR-3727' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-3728' ha

Row with Unique_ID 'IR-3856' has been updated.
Row with Unique_ID 'IR-3857' has been updated.
Row with Unique_ID 'IR-3858' has been updated.
Row with Unique_ID 'IR-3859' has been updated.
Iteration: 570, Elapsed Time: 234 minutes and 23.38 seconds
Row with Unique_ID 'IR-3860' has been updated.
Row with Unique_ID 'IR-3861' has been updated.
Row with Unique_ID 'IR-3862' has been updated.
Row with Unique_ID 'IR-3863' has been updated.
Row with Unique_ID 'IR-3864' has been updated.
Row with Unique_ID 'IR-3865' has been updated.
Row with Unique_ID 'IR-3866' has been updated.
Row with Unique_ID 'IR-3867' has been updated.
Row with Unique_ID 'IR-3868' has been updated.
Row with Unique_ID 'IR-3869' has been updated.
Iteration: 580, Elapsed Time: 236 minutes and 14.18 seconds
Row with Unique_ID 'IR-3870' has been updated.
Row with Unique_ID 'IR-3871' has been updated.
Row with Unique_ID 'IR-3872' has been updated.
Row with Unique_ID 'IR-3873' has been updated.
Row with Unique_ID 'IR-3874' has b

Row with Unique_ID 'IR-4004' has been updated.
Row with Unique_ID 'IR-4005' has been updated.
Row with Unique_ID 'IR-4006' has been updated.
Row with Unique_ID 'IR-4007' has been updated.
Row with Unique_ID 'IR-4008' has been updated.
Row with Unique_ID 'IR-4009' has been updated.
Iteration: 720, Elapsed Time: 259 minutes and 23.56 seconds
Row with Unique_ID 'IR-4010' has been updated.
Row with Unique_ID 'IR-4011' has been updated.
Row with Unique_ID 'IR-4012' has been updated.
Row with Unique_ID 'IR-4013' has been updated.
Row with Unique_ID 'IR-4014' has been updated.
Row with Unique_ID 'IR-4015' has been updated.
Row with Unique_ID 'IR-4016' has been updated.
Row with Unique_ID 'IR-4017' has been updated.
Row with Unique_ID 'IR-4018' has been updated.
Row with Unique_ID 'IR-4019' has been updated.
Iteration: 730, Elapsed Time: 261 minutes and 0.36 seconds
Row with Unique_ID 'IR-4020' has been updated.
Row with Unique_ID 'IR-4021' has been updated.
Row with Unique_ID 'IR-4022' has be

Row with Unique_ID 'IR-4148' has been updated.
Row with Unique_ID 'IR-4149' has been updated.
Iteration: 860, Elapsed Time: 283 minutes and 55.80 seconds
Row with Unique_ID 'IR-4150' has been updated.
Row with Unique_ID 'IR-4151' has been updated.
Row with Unique_ID 'IR-4152' has been updated.
Row with Unique_ID 'IR-4153' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-4154' has been updated.
Row with Unique_ID 'IR-4155' has been updated.
Error encountered: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.. Retry 1/5
Error encountered: The `response.text` quick accessor only works when the response contains a valid `Part`, but none was returned. Check the `candidate.safety_ratings` to see if the response was blocked.. Retry 2/5
Row with Unique_ID 'IR-4156' has been updated.
Row with Unique_ID 

Row with Unique_ID 'IR-4283' has been updated.
Row with Unique_ID 'IR-4284' has been updated.
Row with Unique_ID 'IR-4285' has been updated.
Row with Unique_ID 'IR-4286' has been updated.
Row with Unique_ID 'IR-4287' has been updated.
Row with Unique_ID 'IR-4288' has been updated.
Row with Unique_ID 'IR-4289' has been updated.
Iteration: 1000, Elapsed Time: 309 minutes and 15.44 seconds
Row with Unique_ID 'IR-4290' has been updated.
Row with Unique_ID 'IR-4291' has been updated.
Row with Unique_ID 'IR-4292' has been updated.
Row with Unique_ID 'IR-4293' has been updated.
Row with Unique_ID 'IR-4294' has been updated.
Row with Unique_ID 'IR-4295' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Error encountered: 'str' object has no attribute 'items'. Retry 2/5
Row with Unique_ID 'IR-4296' has been updated.
Row with Unique_ID 'IR-4297' has been updated.
Row with Unique_ID 'IR-4298' has been updated.
Row with Unique_ID 'IR-4299' has been updated.
Iter

Row with Unique_ID 'IR-4434' has been updated.
Row with Unique_ID 'IR-4435' has been updated.
Row with Unique_ID 'IR-4436' has been updated.
Row with Unique_ID 'IR-4437' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'IR-4438' has been updated.
Row with Unique_ID 'IR-4439' has been updated.
Iteration: 1150, Elapsed Time: 335 minutes and 38.72 seconds
Row with Unique_ID 'IR-4440' has been updated.
Row with Unique_ID 'IR-4441' has been updated.
Row with Unique_ID 'IR-4442' has been updated.
Row with Unique_ID 'IR-4443' has been updated.
Row with Unique_ID 'IR-4444' has been updated.
Row with Unique_ID 'IR-4445' has been updated.
Row with Unique_ID 'IR-4446' has been updated.
Row with Unique_ID 'IR-4447' has been updated.
Row with Unique_ID 'IR-4448' has been updated.
Row with Unique_ID 'IR-4449' has been updated.
Iteration: 1160, Elapsed Time: 337 minutes and 32.94 seconds
Row with Unique_ID 'IR-4450' has been updated.
Row with Un

Row with Unique_ID 'IR-4582' has been updated.
Row with Unique_ID 'IR-4583' has been updated.
Row with Unique_ID 'IR-4584' has been updated.
Row with Unique_ID 'IR-4585' has been updated.
Row with Unique_ID 'IR-4586' has been updated.
Row with Unique_ID 'IR-4587' has been updated.
Row with Unique_ID 'IR-4588' has been updated.
Row with Unique_ID 'IR-4589' has been updated.
Iteration: 1300, Elapsed Time: 364 minutes and 35.48 seconds
Row with Unique_ID 'IR-4590' has been updated.
Row with Unique_ID 'IR-4591' has been updated.
Row with Unique_ID 'IR-4592' has been updated.
Row with Unique_ID 'IR-4593' has been updated.
Row with Unique_ID 'IR-4594' has been updated.
Row with Unique_ID 'IR-4595' has been updated.
Row with Unique_ID 'IR-4596' has been updated.
Row with Unique_ID 'IR-4597' has been updated.
Row with Unique_ID 'IR-4598' has been updated.
Row with Unique_ID 'IR-4599' has been updated.
Iteration: 1310, Elapsed Time: 366 minutes and 32.81 seconds
Row with Unique_ID 'IR-4600' has